In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
model = open(r'C:\Users\KIIT\Documents\LGM-Soc contributions\Airline Passenger Satisfaction\Model/hypertuned_rf.pkl','rb')
rf_tuned = pickle.load(model)

In [3]:
model = open(r'C:\Users\KIIT\Documents\LGM-Soc contributions\Airline Passenger Satisfaction\Model/hypertuned_svc.pkl','rb')
svc_tuned = pickle.load(model)

In [4]:
test_df = pd.read_csv(r'C:\Users\KIIT\Documents\LGM-Soc contributions\Airline Passenger Satisfaction\Dataset/test.csv')

In [5]:
test_df.drop(['id'],axis=1,inplace=True)

In [6]:
test_df

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Female,Loyal Customer,52,Business travel,Eco,160,5,4,3,4,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,Female,Loyal Customer,36,Business travel,Business,2863,1,1,3,1,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,Male,disloyal Customer,20,Business travel,Eco,192,2,0,2,4,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,Male,Loyal Customer,44,Business travel,Business,3377,0,0,0,2,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,4,3,...,2,2,2,2,4,2,4,0,20.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25971,Male,disloyal Customer,34,Business travel,Business,526,3,3,3,1,...,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
25972,Male,Loyal Customer,23,Business travel,Business,646,4,4,4,4,...,4,4,5,5,5,5,4,0,0.0,satisfied
25973,Female,Loyal Customer,17,Personal Travel,Eco,828,2,5,1,5,...,2,4,3,4,5,4,2,0,0.0,neutral or dissatisfied
25974,Male,Loyal Customer,14,Business travel,Business,1127,3,3,3,3,...,4,3,2,5,4,5,4,0,0.0,satisfied


In [7]:
test_df['satisfaction'] = test_df.satisfaction.apply(lambda x: int(1) if x == 'satisfied' else int(0))

In [8]:
columns = [column for column in test_df.columns if test_df[column].isnull().sum() > 0]

In [9]:
num_columns = [column for column in test_df.columns if test_df[column].dtypes != 'O']

In [10]:
num_columns_discrete = [column for column in num_columns if len(test_df[column].unique()) <= 10 and column not in ['satisfaction']]

In [11]:
num_columns_continue = [column for column in num_columns if column not in num_columns_discrete and column not in ['satisfaction']] 

In [12]:
for col in num_columns_discrete:
    modus_value = str(test_df[col].mode()[0])
    test_df[col] = test_df[col].astype(str).apply(lambda x: x.replace('0', modus_value))
    test_df[col] = test_df[col].astype(int)

In [13]:
num_columns_continue = [column for column in num_columns if column not in num_columns_discrete and column not in ['satisfaction']] 

In [14]:
cat_columns = [column for column in test_df.columns if column not in num_columns]

In [15]:
for col in columns:
    median_value = test_df[col].median()
    test_df[col] = test_df[col].fillna(median_value)

In [16]:
for col in num_columns_discrete:
    modus_value = str(test_df[col].mode()[0])
    test_df[col] = test_df[col].astype(str).apply(lambda x: x.replace('0', modus_value))
    test_df[col] = test_df[col].astype(int)

In [17]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()

In [18]:
def encode_category(data, column, target):
    ordinal_data= data.groupby([column])[target].sum().sort_values(by=column, ascending=False).index
    ordinal_num = {k: i for i, k in enumerate(ordinal_data, start=0)}
    data[column] = data[column].map(ordinal_num)
    print(ordinal_data)

In [19]:
for col in cat_columns:
    encode_category(test_df, col, ['satisfaction'])

Index(['Male', 'Female'], dtype='object', name='Gender')
Index(['disloyal Customer', 'Loyal Customer'], dtype='object', name='Customer Type')
Index(['Personal Travel', 'Business travel'], dtype='object', name='Type of Travel')
Index(['Eco Plus', 'Eco', 'Business'], dtype='object', name='Class')


In [20]:
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler()
columns_training = [column for column in test_df.columns if column not in ['satisfaction']]
test_df[columns_training] = sc_X.fit_transform(test_df[columns_training])

In [21]:
test_data = test_df.drop(['satisfaction'], axis=1)

In [22]:
pd.set_option('display.max_columns', None)

In [23]:
test_data.drop(['Gender','Departure/Arrival time convenient','Departure Delay in Minutes','Arrival Delay in Minutes'],axis=1,inplace=True)

In [24]:
test_data

,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness
0,1.0,0.576923,1.0,0.5,0.026050,1.00,0.50,0.75,0.50,0.75,0.50,1.00,1.00,1.00,1.00,0.25,1.00,1.00
1,1.0,0.371795,1.0,1.0,0.571890,0.00,0.50,0.00,1.00,0.75,1.00,0.75,0.75,0.75,0.75,0.50,0.75,1.00
2,0.0,0.166667,1.0,0.5,0.032512,0.25,0.25,0.75,0.25,0.25,0.25,0.25,0.75,0.00,0.50,0.25,0.25,0.25
3,1.0,0.474359,1.0,1.0,0.675687,0.25,0.25,0.25,0.50,0.75,0.75,0.00,0.00,0.00,0.00,0.50,0.00,0.75
4,1.0,0.538462,1.0,0.5,0.232431,0.25,0.75,0.50,0.75,0.00,0.25,0.25,0.25,0.25,0.25,0.75,0.25,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25971,0.0,0.346154,1.0,1.0,0.099960,0.50,0.50,0.00,0.75,0.50,0.75,0.75,0.50,0.25,0.75,0.75,1.00,0.75
25972,1.0,0.205128,1.0,1.0,0.124192,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,1.00,1.00,1.00,1.00,0.75
25973,1.0,0.128205,0.0,0.5,0.160945,0.25,0.00,1.00,0.25,0.00,0.25,0.25,0.75,0.50,0.75,1.00,0.75,0.25
25974,1.0,0.089744,1.0,1.0,0.221325,0.50,0.50,0.50,0.75,0.75,0.75,0.75,0.50,0.25,1.00,0.75,1.00,0.75


In [25]:
test_data.shape

(25976, 18)

In [26]:
rf_preds = rf_tuned.predict(test_data)

In [27]:
rf_preds

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [31]:
rf_predictions = pd.DataFrame()
rf_predictions['Predicted satisfaction'] = rf_preds
rf_predictions

,Predicted satisfaction
0,0
1,1
2,0
3,0
4,0
...,...
25971,0
25972,0
25973,0
25974,0


In [32]:
rf_predictions['Predicted satisfaction']  = rf_predictions['Predicted satisfaction'].apply(str)

In [33]:
rf_predictions.replace(['0','1']
                ,['satisfied','neutral or dissatisfied'],inplace=True)


rf_predictions.head(5)

,Predicted satisfaction
0,satisfied
1,neutral or dissatisfied
2,satisfied
3,satisfied
4,satisfied


In [34]:
rf_predictions.to_csv(r'C:\Users\KIIT\Documents\LGM-Soc contributions\Airline Passenger Satisfaction\Dataset/hypertuned rf predictions.csv',header=True,index=False)

In [28]:
svc_preds = svc_tuned.predict(test_data)

In [29]:
svc_preds

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [35]:
svc_predictions = pd.DataFrame()
svc_predictions['Predicted satisfaction'] = svc_preds
svc_predictions

,Predicted satisfaction
0,0
1,0
2,0
3,0
4,1
...,...
25971,0
25972,0
25973,0
25974,0


In [36]:
svc_predictions['Predicted satisfaction']  = svc_predictions['Predicted satisfaction'].apply(str)

In [37]:
svc_predictions.replace(['0','1']
                ,['satisfied','neutral or dissatisfied'],inplace=True)


svc_predictions.head(5)

,Predicted satisfaction
0,satisfied
1,satisfied
2,satisfied
3,satisfied
4,neutral or dissatisfied


In [38]:
svc_predictions.to_csv(r'C:\Users\KIIT\Documents\LGM-Soc contributions\Airline Passenger Satisfaction\Dataset/hypertuned svc predictions.csv',header=True,index=False)